In [ ]:
import os 
! kaggle datasets download -d grassknoted/asl-alphabet

100% 1.02G/1.03G [00:22<00:00, 43.1MB/s]
100% 1.03G/1.03G [00:23<00:00, 47.8MB/s]


In [ ]:
import zipfile
zip=zipfile.ZipFile('/content/asl-alphabet.zip')

zip.extractall()
zip.close()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPooling2D,MaxPool2D,Dropout,BatchNormalization
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
#tf.keras.layers.Dense()

In [ ]:
train=image_dataset_from_directory('asl_alphabet_train/asl_alphabet_train',
                                   label_mode='categorical',
                                   image_size=(224,224),
                                   seed=42,
                                   validation_split=0.2, 
                                   subset='training')
test=image_dataset_from_directory('asl_alphabet_train/asl_alphabet_train',
                                   label_mode='categorical',
                                   image_size=(224,224),
                                   seed=42,
                                  validation_split=0.2, 
                                  subset='validation')

Found 87000 files belonging to 29 classes.
Using 69600 files for training.
Found 87000 files belonging to 29 classes.
Using 17400 files for validation.


In [ ]:
model=Sequential([
    Conv2D(16,3,activation='relu',input_shape=(224,224,3)),
    MaxPool2D(),
    Conv2D(32,3,activation='relu'),
    MaxPool2D(),
    Conv2D(32,3,activation='relu'),
    MaxPool2D(),
    Conv2D(64,3,activation='relu'),
    MaxPool2D(),

    Flatten(),
    Dense(128,activation='relu'),
    Dense(64,activation='relu'),
    Dense(29,activation='softmax')
    
])

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adagrad(),
                metrics=['accuracy'])

In [ ]:
model.fit(train,epochs=10,
          steps_per_epoch=(len(train)),
          validation_data=test,
          validation_steps=(0.1*len(test)))

Epoch 1/10
2175/2175 [==============================] - 146s 67ms/step - loss: 2.4313 - accuracy: 0.3185 - val_loss: 1.4176 - val_accuracy: 0.5972
Epoch 2/10
2175/2175 [==============================] - 145s 66ms/step - loss: 0.9598 - accuracy: 0.7146 - val_loss: 0.7315 - val_accuracy: 0.7932
Epoch 3/10
2175/2175 [==============================] - 145s 67ms/step - loss: 0.5191 - accuracy: 0.8498 - val_loss: 0.4874 - val_accuracy: 0.8517
Epoch 4/10
2175/2175 [==============================] - 146s 67ms/step - loss: 0.3320 - accuracy: 0.9065 - val_loss: 0.3663 - val_accuracy: 0.8909
Epoch 5/10
2175/2175 [==============================] - 145s 67ms/step - loss: 0.2307 - accuracy: 0.9378 - val_loss: 0.2620 - val_accuracy: 0.9290
Epoch 6/10
2175/2175 [==============================] - 146s 67ms/step - loss: 0.1676 - accuracy: 0.9576 - val_loss: 0.2249 - val_accuracy: 0.9312
Epoch 7/10
2175/2175 [==============================] - 146s 67ms/step - loss: 0.1271 - accuracy: 0.9695 - val_loss: 0

In [ ]:
model.evaluate(test)

544/544 [==============================] - 27s 49ms/step - loss: 0.0909 - accuracy: 0.9760


[0.09087515622377396, 0.9760344624519348]

In [ ]:
te=image_dataset_from_directory('test/asl_alphabet_test',
                                   label_mode='categorical',
                                   image_size=(224,224),
                                   seed=42)
model.evaluate(te)

Found 29 files belonging to 29 classes.
1/1 [==============================] - 0s 89ms/step - loss: 18.7238 - accuracy: 0.4828


[18.723796844482422, 0.48275861144065857]

In [ ]:
model.save('sign_images.h5')

In [ ]:
preprocess=Sequential([
      tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
])

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow as tf
base=tf.keras.applications.InceptionResNetV2(include_top=False)
base.trainable=False
inputs=tf.keras.layers.Input(shape=(224,224,3))
x=preprocess(inputs)
x=base(x,training=False)

x=tf.keras.layers.GlobalAvgPool2D()(x)
output=tf.keras.layers.Dense(29,activation='softmax')(x)
model1=tf.keras.Model(inputs,output)

In [ ]:
model1.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

history=model1.fit(train,epochs=5,
          steps_per_epoch=(0.1*len(train)),
          validation_data=test,
          validation_steps=(0.1*len(test)))


Epoch 1/5
217/217 [==============================] - 123s 509ms/step - loss: 1.9044 - accuracy: 0.4994 - val_loss: 1.1360 - val_accuracy: 0.7284
Epoch 2/5
217/217 [==============================] - 107s 495ms/step - loss: 0.9556 - accuracy: 0.7729 - val_loss: 0.7494 - val_accuracy: 0.8335
Epoch 3/5
217/217 [==============================] - 108s 495ms/step - loss: 0.6868 - accuracy: 0.8438 - val_loss: 0.5720 - val_accuracy: 0.8744
Epoch 4/5
217/217 [==============================] - 107s 494ms/step - loss: 0.5481 - accuracy: 0.8727 - val_loss: 0.4878 - val_accuracy: 0.8858
Epoch 5/5
217/217 [==============================] - 108s 495ms/step - loss: 0.4621 - accuracy: 0.8925 - val_loss: 0.3966 - val_accuracy: 0.9045


In [ ]:
model1.save('transfer_asl.h5')

In [ ]:
base.trainable = True

# Freeze all layers except for the
for layer in base.layers[:-10]:
  layer.trainable = False

# Recompile the model (always recompile after any adjustments to a model)
model1.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(lr=0.0001), # lr is 10x lower than before for fine-tuning
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
fine_tune_epochs = 5 + 5

# Refit the model (same as model_2 except with more trainable layers)
history_fine_10_percent_data_aug = model1.fit(train,
                                               epochs=fine_tune_epochs,
                                               steps_per_epoch=(0.1*(len(train))),
                                               validation_data=test,
                                               initial_epoch=history.epoch[-1], # start from previous last epoch
                                               validation_steps=int(0.25 * len(test)))

Epoch 5/10
217/217 [==============================] - 177s 664ms/step - loss: 0.2348 - accuracy: 0.9365 - val_loss: 0.1804 - val_accuracy: 0.9508
Epoch 6/10
217/217 [==============================] - 141s 647ms/step - loss: 0.1295 - accuracy: 0.9680 - val_loss: 0.0894 - val_accuracy: 0.9786
Epoch 7/10
217/217 [==============================] - 141s 648ms/step - loss: 0.0852 - accuracy: 0.9786 - val_loss: 0.0667 - val_accuracy: 0.9853
Epoch 8/10
217/217 [==============================] - 140s 647ms/step - loss: 0.0692 - accuracy: 0.9837 - val_loss: 0.0567 - val_accuracy: 0.9862
Epoch 9/10
217/217 [==============================] - 140s 646ms/step - loss: 0.0663 - accuracy: 0.9834 - val_loss: 0.0535 - val_accuracy: 0.9855
Epoch 10/10
217/217 [==============================] - 141s 647ms/step - loss: 0.0517 - accuracy: 0.9860 - val_loss: 0.0395 - val_accuracy: 0.9908


In [ ]:
model1.save('asl_transfer.h5')